In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

In [2]:
# load the csv data file
df = pd.read_csv("nci60.csv")
# X = np.array(df.iloc[:,2:])
# y = np.array(df.iloc[:,1])
X = df.iloc[:,2:]
y = df.iloc[:,1]

In [3]:
def rfe_rf_tim(X, y, num_feature_to_selected):
    from sklearn.ensemble import RandomForestRegressor
    import time
    rf = RandomForestRegressor()
    X_new = X
    record = {}
    keep_ = X.shape[1]-num_feature_to_selected
    start_time = time.time()
    for i in range(keep_):
        # fit the random forest and get the weights of features
        fit = rf.fit(X_new, y)
        # remove the feature with the smallest weight
        X_new = X_new.drop(X_new.columns[[np.argmin(fit.feature_importances_)]], 1)
        # record the index and value of the deleted feature
        record.update({str(np.argmin(fit.feature_importances_)):fit.feature_importances_[np.argmin(fit.feature_importances_)]})
    elapsed_time = time.time() - start_time
    return X_new, record, elapsed_time

In [5]:
X_new, record, elapsed_time = rfe_rf_tim(X, y, 25)
# for key,value in record.items(): 
#     print(key,':',value)

X_new.shape[1]
elapsed_time

2191.6778178215027

In [8]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=5)
rf = RandomForestRegressor()
# model = clf.set_params(kernel='linear')
score = cross_val_score(rf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

-0.82997179637651686

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
k_fold = KFold(n_splits=5)
clf = SVR()
# model = clf.set_params(kernel='linear')
score = cross_val_score(clf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

-0.39388203872926386